In [1]:
import pandas as pd
import numpy as np

### Non-contextual bandit problem (trivial state).

Consider the sub-problem where rewards depend on action but not state. In this case there is a single best action for everyone (no personalization opportunity).

You're running a website for an artist who has four products. You're making a popup with a special offer to sign up for a mailing list, which offer should you choose?

In [67]:
def get_noncon_reward(action):
    if action == 'a1':
        return np.random.binomial(1, 0.03)
    elif action == 'a2':
        return np.random.binomial(1, 0.02)
    elif action == 'a3' or action == 'a4':
        return np.random.binomial(1, 0.01)
    else:
        raise ValueError('Unrecognized action.')

In [68]:
num_sessions = 100000

#### Try number 1 - random serving:

In [71]:
rewards = []

for i in range(num_sessions):
    action = np.random.choice(['a1', 'a2', 'a3', 'a4'])
    rewards.append(get_noncon_reward(action))
    
cumulative_reward = np.array(rewards).sum()
print('Profit: ', cumulative_reward)

Profit:  1797


#### Try number 2 - explore first:

In [115]:
exploration_budget = num_sessions/4
learned = False
rewards = []
history = {'a1': [], 'a2': [], 'a3': [], 'a4': []}

for i in range(num_sessions):
    if (i < exploration_budget): #explore here!
        action = np.random.choice(['a1', 'a2', 'a3', 'a4'])
        reward = get_noncon_reward(action)
        rewards.append(reward)
        # add to history
        history[action].append(reward)
    else:
        if not learned:
            #learn!
            best_rate = 0.0
            best_action = 'unknown'
            for a in history:
                rate = np.array(history[a]).mean()
                if rate > best_rate:
                    best_rate = rate
                    best_action = a
            learned = True
            rewards.append(get_noncon_reward(best_action))
        else:
            #be greedy!
            rewards.append(get_noncon_reward(best_action))
            
cumulative_reward = np.array(rewards).sum()
print('Profit: ', cumulative_reward)
print('Best learned action: ', best_action)

Profit:  2559
Best learned action:  a1


#### Thompson Sampling (simple but optimal bandit algorithm)

In [121]:
rewards = []
successes = [1,1,1,1]
failures = [1,1,1,1]
action_dict = {0: 'a1', 1: 'a2', 2: 'a3', 3: 'a4'}

for i in range(num_sessions):
    beta_samples = np.random.beta(successes, failures)
    action_index = np.argmax(beta_samples)
    action = action_dict[action_index]
    reward = get_noncon_reward(action)
    rewards.append(reward)
    #learn
    successes[action_index] += reward
    failures[action_index] += 1-reward
    
    
cumulative_reward = np.array(rewards).sum()
print('Profit: ', cumulative_reward)

Profit:  2956
